Notebook responsável pela criação da tabela dimensão relacionada ao fornecedor dos dados (VendorID).

Informações retiradas das documentações em:

https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf
https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_green.pdf

In [0]:
import sys
import os
sys.path.append('../utils') 

from utils import create_update_table_metadata
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [0]:
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()

folder_path = os.path.dirname(notebook_path)

In [0]:
schema = T.StructType([
    T.StructField("id_vendor", T.IntegerType(), nullable=False),
    T.StructField("ds_vendor_name", T.StringType(), nullable=True)
])

vendor_data = [
    (1, "Creative Mobile Technologies, LLC"),
    (2, "VeriFone Inc.")
]

table_name = "tb_dim_vendor"
silver_table_name = f"ifood_case.silver.{table_name}"

df_silver = spark.createDataFrame(vendor_data, schema)

In [0]:
df_silver.write.format("delta").mode("overwrite").saveAsTable(f"{silver_table_name}")

In [0]:
metadata_file_path = f"/Workspace/{folder_path}/metadata/{table_name}.yaml"
create_update_table_metadata(spark, silver_table_name, metadata_file_path)